In [ ]:
import feedparser
import openai
import requests
import json
import urllib.parse
import random
import time
import os # ←【追加】クラウドの金庫を開ける鍵

# ==========================================
# 🔍 検索設定
# ==========================================
KEYWORDS = [
    "企業 新規事業",
    "業務提携 戦略", 
    "DX 導入事例",
    "マーケティング戦略",
    "スタートアップ 資金調達"
]

# ★クラウドの「金庫（Secrets）」から鍵を取り出す設定
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY") 
SLACK_WEBHOOK_URL = os.environ.get("SLACK_WEBHOOK_URL")

def get_top10_news():
    selected_keyword = random.choice(KEYWORDS)
    print(f"🔍 テーマ「{selected_keyword}」でニュースを10件収集中...\n")
    
    encoded_keyword = urllib.parse.quote(selected_keyword)
    rss_url = f"https://news.google.com/rss/search?q={encoded_keyword}+when:7d&hl=ja&gl=JP&ceid=JP:ja"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get(rss_url, headers=headers, timeout=10)
        feed = feedparser.parse(response.content)
        return feed.entries[:10], selected_keyword
    except Exception as e:
        print(f"接続エラー: {e}")
        return [], selected_keyword

def analyze_business_model(news_item, keyword):
    # APIキーがない場合（模擬モード）
    if not OPENAI_API_KEY or not OPENAI_API_KEY.startswith("sk-"):
        mock_analyses = [
            "【模擬分析】これはテストです。本番キーを設定するとAI分析が表示されます。",
            "【模擬分析】面接ネタ：このニュースの背景には人手不足があります。",
        ]
        return random.choice(mock_analyses)

    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    prompt = f"""
    記事タイトル: {news_item['title']}
    検索テーマ: {keyword}
    就活生向けに、この記事の「ビジネス的な狙い」と「面接で使える意見」を100文字以内でまとめて。
    """
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"分析エラー: {e}"

def send_slack(text, article, keyword):
    if not SLACK_WEBHOOK_URL:
        print("⚠️ Slack URL未設定のためスキップ")
        return

    payload = {
        "text": f"👔 ビジネスニュース速報（テーマ: {keyword}）\n\n{text}\n\n🔗 <{article.link}|元記事を確認する>",
        "username": "戦略分析bot",
        "icon_emoji": ":chart_with_upwards_trend:"
    }
    requests.post(SLACK_WEBHOOK_URL, data=json.dumps(payload, ensure_ascii=False).encode('utf-8'))

# --- メイン実行部 ---
news_list, keyword = get_top10_news()
if news_list:
    print(f"✅ {len(news_list)}件のニュースが見つかりました。")
    for i, news in enumerate(news_list):
        analysis = analyze_business_model(news, keyword)
        send_slack(analysis, news, keyword)
        time.sleep(1) 
else:
    print("ニュースなし")